In [3]:
%pip install plotly pandas
%pip install --upgrade nbformat
%pip install nbformat
%pip install plotly
%pip install -U kaleido

import warnings
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
warnings.filterwarnings("ignore", message=".*scattermapbox.*", category=DeprecationWarning)


df = pd.read_csv("filtered_main_data.csv")
df_clean = df[['longitude', 'latitude', 'poverty population', 'CensusTract']].dropna()

coords = df_clean[['longitude', 'latitude']].to_numpy()
weights = df_clean['poverty population'].to_numpy()

k = int(input("Enter the number of facilities to place: "))
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(coords, sample_weight=weights)

df_clean['cluster'] = kmeans.predict(coords)
df['cluster'] = df_clean['cluster']

closest_indices, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, coords)
selected_tracts = df_clean.iloc[closest_indices]['CensusTract'].values
centers = kmeans.cluster_centers_

print("\nFacility locations (closest CensusTract IDs):")
for i, tract in enumerate(selected_tracts, start=1):
    print(f"Facility {i}: CensusTract {tract}")

facilities = pd.DataFrame({
    'CensusTract': selected_tracts,
    'longitude': centers[:, 0],
    'latitude': centers[:, 1]
})
gdf_facilities = gpd.GeoDataFrame(
    facilities,
    geometry=gpd.points_from_xy(facilities['longitude'], facilities['latitude']),
    crs="EPSG:4326"
).to_crs(epsg=3857)


fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "mapbox", "colspan": 2}, None],
           [{"type": "mapbox"}, {"type": "mapbox"}]],
    row_heights=[0.5, 0.5],
    vertical_spacing=0.005,
    column_widths=[0.5, 0.5],
    subplot_titles=["Continental US", "Alaska", "Hawaii"]
)

for i, row in facilities.iterrows():
    fig.add_trace(
        go.Scattermapbox(
            lat=[row['latitude']],
            lon=[row['longitude']],
            mode='markers',
            marker=dict(size=10, color='#58B8E4'),
            name=f"Facility {i+1}",
            hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=1, col=1  # All facilities shown on US map
    )
    fig.add_trace(
        go.Scattermapbox(
            lat=[row['latitude']],
            lon=[row['longitude']],
            mode='markers',
            marker=dict(size=10, color='#58B8E4'),    
            name=f"Facility {i+1}",
            hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=2, col=1
    )
        
    fig.add_trace(
        go.Scattermapbox(
        lat=[row['latitude']],
        lon=[row['longitude']],
        mode='markers',
        marker=dict(size=10, color='#58B8E4'),
        name=f"Facility {i+1}",
        hovertext=f"CensusTract {row['CensusTract']}"
        ),
        row=2, col=2
    )

fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center=dict(lat=37.5, lon=-95),
        zoom=2.5
    ),
    mapbox2=dict(
        style="carto-positron",
        center=dict(lat=64, lon=-150),
        zoom=1.5
    ),
    mapbox3=dict(
        style="carto-positron",
        center=dict(lat=20.5, lon=-157.5),
        zoom=4
    ),
    mapbox_domain=dict(row=1, column=1),
    mapbox2_domain=dict(row=2, column=1),
    mapbox3_domain=dict(row=2, column=2),
    height=800,
    title_text="Facility Placement Map: US, Alaska, and Hawaii",
    showlegend=False
)

fig.show()
facilities.to_csv(f'{k} optimal_facility_locations.csv', index=False)
print(f"\n✅ Saved {k} facility locations to 'optimal_facility_locations.csv'")

/var/folders/rn/gtj4kyrs66s7lwh782tpf_bw0000gn/T/ipykernel_3191/2078992452.py:4: DtypeWarning:

Columns (0,8,9) have mixed types. Specify dtype option on import or set low_memory=False.




Facility locations (closest CensusTract IDs):
Facility 1: CensusTract 39061003800.0
Facility 2: CensusTract 29009960500.0
Facility 3: CensusTract 6007001702.0
Facility 4: CensusTract 24033802700.0
Facility 5: CensusTract 48215023504.0
Facility 6: CensusTract 13089023313.0
Facility 7: CensusTract 28089030500.0
Facility 8: CensusTract 8077001706.0
Facility 9: CensusTract 19057000200.0
Facility 10: CensusTract 12101031807.0
Facility 11: CensusTract 33017083001.0
Facility 12: CensusTract 26121001300.0
Facility 13: CensusTract 39007000500.0
Facility 14: CensusTract 27137012200.0
Facility 15: CensusTract 6071010901.0
Facility 16: CensusTract 48143950500.0
Facility 17: CensusTract 16065950302.0
Facility 18: CensusTract 31109002701.0
Facility 19: CensusTract 35029000400.0
Facility 20: CensusTract 37027030800.0
Facility 21: CensusTract 41019130000.0
Facility 22: CensusTract 22071007501.0
Facility 23: CensusTract 17163502800.0
Facility 24: CensusTract 1073010602.0
Facility 25: CensusTract 39139


✅ Saved 992 facility locations to 'optimal_facility_locations.csv'
